In [11]:
import spacy
import numpy as np
import pandas as pd
from stopwords import ENGLISH_STOP_WORDS
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [2]:
en_nlp = spacy.en.English()

In [3]:
def spacy_get_vec(sentence):
    vec = np.zeros(300)
    doc = en_nlp((sentence))
    for word in doc:
        #if word.lower_ in ENGLISH_STOP_WORDS:
        #    continue
        vec += word.vector
    return vec

In [106]:
lines = open('./question.txt').readlines()
vecs = []
intents = []
idfs = []
for line in lines:
    tokens = line.split(',')
    sentence = tokens[0]
    intent = tokens[1]
    if intent[-1] == '\n':
        intent = intent[:-1]
    vecs.append(spacy_get_vec(sentence))
    intents.append(intent)
    #idfs.append(get_idf(sentence))

df = pd.DataFrame(vecs, columns=['vec_%d' % i for i in range(300)])
#df['idf'] = idfs
df['intents'] = intents
df.intents = df.intents.astype('category')

In [107]:
df.head()

,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,vec_9,...,vec_291,vec_292,vec_293,vec_294,vec_295,vec_296,vec_297,vec_298,vec_299,intents
0,-0.058150,0.004059,-0.145796,-0.061884,0.031343,0.087341,-0.194867,-0.137436,0.133669,0.032237,...,-0.200298,-0.014908,-0.105034,0.027138,-0.100345,-0.239374,0.153825,0.109905,-0.210085,name
1,0.054641,0.101480,-0.140236,-0.209629,0.210244,0.237446,-0.133493,0.125002,0.169063,0.036991,...,-0.113492,0.112089,0.050605,-0.036360,-0.207701,-0.160545,0.115035,-0.239938,-0.125308,name
2,-0.044503,0.003799,-0.173259,-0.198783,0.137944,0.098054,-0.085885,0.017625,0.202091,0.082392,...,-0.061114,0.130471,0.045094,0.003426,-0.230525,-0.097541,0.128447,-0.085151,-0.083890,name
3,0.144701,0.116740,0.015040,-0.169500,-0.038329,0.168020,-0.264787,-0.123588,0.036654,0.010474,...,-0.007536,0.008177,-0.227186,0.108725,-0.369863,-0.225025,0.265710,0.031394,-0.013227,name
4,0.211440,0.160685,-0.051632,-0.143768,-0.004933,0.070291,-0.258258,-0.179165,0.088886,-0.110131,...,-0.053267,0.092519,-0.351504,0.118517,-0.268701,-0.147621,0.174754,0.017807,-0.075447,name


In [108]:
from sklearn.utils import shuffle
df = shuffle(df)

In [109]:
X = df.iloc[:, :-1].values
y = df.iloc[:,-1:].values.ravel()

In [110]:
from sklearn.linear_model import LogisticRegression
logit_model = LogisticRegression()
logit_model.fit(X, y)
print(logit_model.score(X, y))


0.943396226415


In [111]:
from sklearn.ensemble import GradientBoostingClassifier
gradboost = GradientBoostingClassifier(n_estimators=600, max_depth=30, learning_rate=0.01)
gradboost.fit(X, y)
print(gradboost.score(X, y))


1.0


In [113]:
sent = 'do you live in the computer'
print(gradboost.predict_log_proba(spacy_get_vec(sent)))
print(gradboost.predict(spacy_get_vec(sent)))

[[-3.4741925  -3.47182637 -0.19479803 -3.17934957 -3.28589624 -3.32611904]]
['location']


In [120]:
sent = 'how does your mom call you'
print(gradboost.predict_log_proba(spacy_get_vec(sent)))
print(gradboost.predict(spacy_get_vec(sent)))

[[ -9.22391919e+00  -9.22155306e+00  -9.02732034e+00  -8.97761287e+00
   -5.58375604e-04  -9.07584573e+00]]
['name']


In [121]:
sent = 'how does my mom call me'
print(gradboost.predict_log_proba(spacy_get_vec(sent)))
print(gradboost.predict(spacy_get_vec(sent)))

[[-4.97554899 -4.97318286 -3.37001435 -2.54032215 -0.14509821 -4.82747553]]
['name']


In [33]:
gradboost.classes_

array(['bot', 'greet_response', 'location', 'marriage', 'name',
       'users_name'], dtype=object)

In [116]:
from sklearn.neural_network import MLPClassifier
nn = MLPClassifier(hidden_layer_sizes=(300,128,64,6), max_iter=1000)

In [117]:
nn.fit(X_train, y_train)
nn.score(X_train, y_train)

1.0

In [119]:
sent = 'how does my mom call me'
#gradboost.predict_proba(np.append(spacy_get_vec(sent), get_idf(sent)))
print(nn.predict_proba(spacy_get_vec(sent)))
print(nn.predict(spacy_get_vec(sent)))

[[  5.60886541e-16   6.20975488e-11   5.84206522e-18   9.99459575e-01
    3.44494571e-16   5.40425181e-04]]
['marriage']


In [122]:
from sklearn.externals import joblib
joblib.dump(gradboost, 'question.pkl')

['question.pkl']